# Transforming data with Pipelines
Pipelines are the construct used to compose filters that transform the published input data into new meshes. This is where users specify typical geometric transforms (e.g., clipping and slicing), field based transforms (e.g., threshold and contour), etc. The resulting data from each Pipeline can be used as input to Scenes or Extracts. Each pipeline contains one or more filters that transform the published mesh data. See Ascent's [Pipelines](https://ascent.readthedocs.io/en/latest/Actions/Pipelines.html) docs for deeper details on Pipelines.

In [ ]:
# cleanup any old results
!./cleanup.sh
# ascent + conduit imports
import conduit
import conduit.blueprint
import ascent

import numpy as np

# Jupyter imports
from IPython.display import Image
# helpers we use when displaying results in the notebook
from ascent_tutorial_jupyter_utils import img_display_width

### Create an example mesh to feed to Ascent

In [ ]:
# create example mesh using the conduit blueprint braid helper
mesh = conduit.Node()
conduit.blueprint.mesh.examples.braid("hexs",
                                      25,
                                      25,
                                      25,
                                      mesh)

## Pipeline Example 1
### Calculating and rendering contours

In [ ]:
a = ascent.Ascent()
a.open()

# publish our mesh to ascent
a.publish(mesh);

# setup actions
actions = conduit.Node()
add_act = actions.append()
add_act["action"] = "add_pipelines"
pipelines = add_act["pipelines"]

# create a  pipeline (pl1) with a contour filter (f1)
pipelines["pl1/f1/type"] = "contour"

# extract contours where braid variable
# equals 0.2 and 0.4
contour_params = pipelines["pl1/f1/params"]
contour_params["field"] = "braid"
iso_vals = np.array([0.2, 0.4],dtype=np.float32)
contour_params["iso_values"].set(iso_vals)

# declare a scene to render the pipeline result

add_act2 = actions.append()
add_act2["action"] = "add_scenes"
scenes = add_act2["scenes"]

# add a scene (s1) with one pseudocolor plot (p1) that 
# will render the result of our pipeline (pl1)
scenes["s1/plots/p1/type"] = "pseudocolor"
scenes["s1/plots/p1/pipeline"] = "pl1"
scenes["s1/plots/p1/field"] = "braid"
# set the output file name (ascent will add ".png")
scenes["s1/image_name"] = "out_pipeline_ex1_contour"

# print our full actions tree
print(actions.to_yaml())

# execute the actions
a.execute(actions)

a.close()

In [ ]:
# show the resulting image
Image("out_pipeline_ex1_contour.png",width=img_display_width())

## Pipeline Example 2:
### Combining threshold and clip transforms

In [ ]:
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

# setup actions
actions = conduit.Node()
add_act = actions.append()
add_act["action"] = "add_pipelines"
pipelines = add_act["pipelines"]

# create a  pipeline (pl1) with a threshold filter (f1)
# and a clip filter (f2)

# add our threshold (f1)
pipelines["pl1/f1/type"] = "threshold"
thresh_params = pipelines["pl1/f1/params"]
# set threshold parameters
# keep elements with values between 0.0 and 0.5
thresh_params["field"]  = "braid"
thresh_params["min_value"] = 0.0
thresh_params["max_value"] = 0.5

# add our clip (f2)
pipelines["pl1/f2/type"]   = "clip"
clip_params = pipelines["pl1/f2/params"]
# set clip parameters
# use spherical clip
clip_params["sphere/center/x"] = 0.0
clip_params["sphere/center/y"] = 0.0
clip_params["sphere/center/z"] = 0.0
clip_params["sphere/radius"]   = 12

#  declare a scene to render the pipeline results
add_act2 = actions.append()
add_act2["action"] = "add_scenes"
scenes = add_act2["scenes"]

# add a scene (s1) with one pseudocolor plot (p1) that 
# will render the result of our pipeline (pl1)
scenes["s1/plots/p1/type"] = "pseudocolor"
scenes["s1/plots/p1/pipeline"] = "pl1"
scenes["s1/plots/p1/field"] = "braid"
# set the output file name (ascent will add ".png")
scenes["s1/image_name"] = "out_pipeline_ex2_thresh_clip"

# print our full actions tree
print(actions.to_yaml())

# execute
a.execute(actions)

# close ascent
a.close()

In [ ]:
# show the resulting image
Image("out_pipeline_ex2_thresh_clip.png",width=img_display_width())

## Pipeline Example 3:
### Creating and rendering multiple pipelines

In [ ]:
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

# setup actions
actions = conduit.Node()
add_act = actions.append()
add_act["action"] = "add_pipelines"
pipelines = add_act["pipelines"]

# create our first pipeline (pl1) 
# with a contour filter (f1)
pipelines["pl1/f1/type"] = "contour"
# extract contours where braid variable
# equals 0.2 and 0.4
contour_params = pipelines["pl1/f1/params"]
contour_params["field"] = "braid"
iso_vals = np.array([0.2, 0.4],dtype=np.float32)
contour_params["iso_values"].set(iso_vals)

# create our second pipeline (pl2) with a threshold filter (f1)
# and a clip filter (f2)

# add our threshold (pl2 f1)
pipelines["pl2/f1/type"] = "threshold"
thresh_params = pipelines["pl2/f1/params"]
# set threshold parameters
# keep elements with values between 0.0 and 0.5
thresh_params["field"]  = "braid"
thresh_params["min_value"] = 0.0
thresh_params["max_value"] = 0.5


# add our clip (pl2 f2)
pipelines["pl2/f2/type"]   = "clip"
clip_params = pipelines["pl2/f2/params"]
# set clip parameters
# use spherical clip
clip_params["sphere/center/x"] = 0.0
clip_params["sphere/center/y"] = 0.0
clip_params["sphere/center/z"] = 0.0
clip_params["sphere/radius"]   = 12

# declare a scene to render our pipeline results
add_act2 = actions.append()
add_act2["action"] = "add_scenes"
scenes = add_act2["scenes"]

# add a scene (s1) with two pseudocolor plots 
# (p1 and p2) that will render the results 
# of our pipelines (pl1 and pl2)## Pipeline Example 2:

# plot (p1) to render our first pipeline (pl1)
scenes["s1/plots/p1/type"] = "pseudocolor"
scenes["s1/plots/p1/pipeline"] = "pl1"
scenes["s1/plots/p1/field"] = "braid"
# plot (p2) to render our second pipeline (pl2)
scenes["s1/plots/p2/type"] = "pseudocolor"
scenes["s1/plots/p2/pipeline"] = "pl2"
scenes["s1/plots/p2/field"] = "braid"
# set the output file name (ascent will add ".png")
scenes["s1/image_name"] = "out_pipeline_ex3_two_plots"

# print our full actions tree
print(actions.to_yaml())

# execute
a.execute(actions)

# close ascent
a.close()

In [ ]:
# show the resulting image
Image("out_pipeline_ex3_two_plots.png",width=img_display_width())